In [21]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [22]:
df1 = pd.read_csv("random_imputed.csv",index_col=[0])


In [23]:
df2_dobu = df1.filter(['systolic.blood.pressure','diastolic.blood.pressure','LVEF','left.ventricular.end.diastolic.diameter.LV'], axis=1)
df2_dobu.head()

,systolic.blood.pressure,diastolic.blood.pressure,LVEF,left.ventricular.end.diastolic.diameter.LV
inpatient.number,,,,
722128,121,56,60.838094,45.000000
723327,150,80,60.838094,4.539744
723617,120,80,60.838094,56.000000
724385,122,86,60.838094,53.000000
725509,95,65,28.000000,72.000000


In [24]:
df3=pd.read_csv("drug_onehot_latest.csv",index_col=[0])
df3.head()

,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
inpatient.number,,,,,,,,,,,,,,,,,,
722128,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
723327,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,0
723617,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
724385,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
725509,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1,0,0


In [25]:
df2=df3['Dobutamine hydrochloride injection']
df2 = df2.to_frame()

In [26]:
df=pd.concat([df2_dobu,df2],axis=1)
df.head()

,systolic.blood.pressure,diastolic.blood.pressure,LVEF,left.ventricular.end.diastolic.diameter.LV,Dobutamine hydrochloride injection
inpatient.number,,,,,
722128,121,56,60.838094,45.000000,0
723327,150,80,60.838094,4.539744,0
723617,120,80,60.838094,56.000000,0
724385,122,86,60.838094,53.000000,0
725509,95,65,28.000000,72.000000,0


In [27]:
cols_to_scale = [df.columns]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_df= scaler.fit_transform(df)

In [28]:
df = pd.DataFrame(scaled_df,columns = df.columns)
df.head()

,systolic.blood.pressure,diastolic.blood.pressure,LVEF,left.ventricular.end.diastolic.diameter.LV,Dobutamine hydrochloride injection
0,0.480159,0.383562,0.679365,0.522336,0.0
1,0.595238,0.547945,0.679365,0.000000,0.0
2,0.476190,0.547945,0.679365,0.664344,0.0
3,0.484127,0.589041,0.679365,0.625614,0.0
4,0.376984,0.445205,0.181818,0.870902,0.0


In [29]:
# Class count
count_class_0, count_class_1 = df['Dobutamine hydrochloride injection'].value_counts()

# Divide by class
df_class_0 = df[df['Dobutamine hydrochloride injection'] == 0]
df_class_1 = df[df['Dobutamine hydrochloride injection'] == 1]

In [31]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over['Dobutamine hydrochloride injection'].value_counts())

Random over-sampling:
0.0    1981
1.0    1981
Name: Dobutamine hydrochloride injection, dtype: int64


In [33]:
X = df_test_over.drop('Dobutamine hydrochloride injection',axis='columns')
y = df_test_over['Dobutamine hydrochloride injection']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [34]:
# Number of classes in training Data
y_train.value_counts()

1.0    1585
0.0    1584
Name: Dobutamine hydrochloride injection, dtype: int64

In [42]:
import tensorflow as tf
from tensorflow import keras



In [44]:
loss = keras.losses.BinaryCrossentropy()
weights = -1


In [52]:
model = keras.Sequential([
    keras.layers.Dense(128, input_dim=4, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])

if weights == -1:
    model.fit(X_train, y_train, epochs=100)
else:
    model.fit(X_train, y_train, epochs=100, class_weight = weights)

print(model.evaluate(X_test, y_test))

y_preds = model.predict(X_test)
y_preds = np.round(y_preds)

print("Classification Report: \n", classification_report(y_test, y_preds))


Epoch 1/100
100/100 [==============================] - 0s 991us/step - loss: 0.6849 - accuracy: 0.5753
Epoch 2/100
100/100 [==============================] - 0s 955us/step - loss: 0.6743 - accuracy: 0.6030
Epoch 3/100
100/100 [==============================] - 0s 877us/step - loss: 0.6643 - accuracy: 0.6056
Epoch 4/100
100/100 [==============================] - 0s 871us/step - loss: 0.6539 - accuracy: 0.6299
Epoch 5/100
100/100 [==============================] - 0s 889us/step - loss: 0.6451 - accuracy: 0.6355
Epoch 6/100
100/100 [==============================] - 0s 887us/step - loss: 0.6376 - accuracy: 0.6314
Epoch 7/100
100/100 [==============================] - 0s 840us/step - loss: 0.6316 - accuracy: 0.6485
Epoch 8/100
100/100 [==============================] - 0s 882us/step - loss: 0.6261 - accuracy: 0.6418
Epoch 9/100
100/100 [==============================] - 0s 892us/step - loss: 0.6213 - accuracy: 0.6523
Epoch 10/100
100/100 [==============================] - 0s 903us/step - l

In [51]:
y_pred = classifier.predict(X_test)